In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\prajwal\Downloads\diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df.corr()['Outcome']  # Impact of all features on Outcome

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
X = scaler.fit_transform(X)

In [9]:
X.shape

(768, 8)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [11]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [12]:
model = Sequential()

model.add(Dense(32, activation = 'relu', input_dim = 8))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

C:\Users\prajwal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.fit(X_train, y_train, batch_size = 32, epochs = 300, validation_data = (X_test, y_test))

Epoch 1/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.4945 - loss: 0.7501 - val_accuracy: 0.5390 - val_loss: 0.7241
Epoch 2/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6375 - loss: 0.6534 - val_accuracy: 0.6104 - val_loss: 0.6587
Epoch 3/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6840 - loss: 0.6018 - val_accuracy: 0.6948 - val_loss: 0.6061
Epoch 4/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7799 - loss: 0.5269 - val_accuracy: 0.7532 - val_loss: 0.5669
Epoch 5/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7581 - loss: 0.5305 - val_accuracy: 0.7662 - val_loss: 0.5405
Epoch 6/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7646 - loss: 0.5136 - val_accuracy: 0.7922 - val_loss: 0.5199
Epoch 7/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7442 - loss: 0.5047 - val_accuracy: 0.8052 - val_loss: 0.5064
Epoch 8/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7628 - loss: 0.4965 - val_accuracy: 0.8052 - 

In [14]:
# Automate hyperparameters
# 1. No. of nodes in a layer
# 2. How to select no. of hidden layers
# 3. All in one Model

In [15]:
!pip install keras-tuner

In [16]:
import keras_tuner as kt

In [17]:
def build_model(hp):
    
    model = Sequential()
    model.add(Dense(32, activation = 'relu', input_dim = 8))
    model.add(Dense(1, activation = 'sigmoid'))
    
    optimizer = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])

    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [18]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5)   # objective will increase or decrese according values according to it

Reloading Tuner from .\untitled_project\tuner0.json


In [19]:
tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

In [20]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [21]:
model = tuner.get_best_models(num_models = 1)[0]

C:\Users\prajwal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.fit(X_train, y_train, batch_size = 32, epochs = 300, initial_epoch = 11)

Epoch 12/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7553 - loss: 0.4972
Epoch 13/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7467 - loss: 0.4908 
Epoch 14/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7722 - loss: 0.4773 
Epoch 15/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7640 - loss: 0.4761 
Epoch 16/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7553 - loss: 0.4791 
Epoch 17/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7772 - loss: 0.4515 
Epoch 18/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7794 - loss: 0.4394 
Epoch 19/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7786 - loss: 0.4505 
Epoch 20/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7650 - loss: 0.4507 
Epoch 21/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7457 - loss: 0.4840 
Epoch 22/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7743 - loss: 0.4497 
Epoch 23/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2

In [24]:
# Now we will be deciding no. of neurons.

In [25]:
def build_model(hp):
    
    model = Sequential()
    units = hp.Int('units', min_value = 8, max_value = 128, step = 8) # will move from 8 to 128  and will return with best result. name 'units' is given by us.

    model.add(Dense(units = units, activation = 'relu', input_dim = 8))
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [26]:
tuner = kt.RandomSearch(build_model,
                       objective = 'val_accuracy',
                       max_trials = 5,
                       directory = 'mydir',
                       project_name = 'myproject')

Reloading Tuner from mydir\myproject\tuner0.json


In [27]:
tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

In [28]:
model = tuner.get_best_models(num_models = 1)[0]

In [29]:
model.fit(X_train, y_train, batch_size = 32, epochs = 300, initial_epoch = 11)

Epoch 12/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7854 - loss: 0.4493   
Epoch 13/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7603 - loss: 0.4733 
Epoch 14/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7531 - loss: 0.4688 
Epoch 15/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7655 - loss: 0.4511 
Epoch 16/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7986 - loss: 0.4237 
Epoch 17/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7896 - loss: 0.4271 
Epoch 18/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7849 - loss: 0.4309 
Epoch 19/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7755 - loss: 0.4478 
Epoch 20/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7884 - loss: 0.4199 
Epoch 21/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7955 - loss: 0.4353 
Epoch 22/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7922 - loss: 0.4289 
Epoch 23/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0

# Now selecting Number of Layers

In [30]:
def build_model(hp):
    
    model = Sequential()
    
    model.add(Dense(72, activation = 'relu', input_dim = 8))
    
    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
        
        model.add(Dense(72, activation = 'relu'))

    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = 'rmsprop', loss='binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [31]:
tuner = kt.RandomSearch(build_model,
                       objective = 'val_accuracy',
                       max_trials = 3,
                       directory = 'mydir',
                       project_name = 'num_layers')

Reloading Tuner from mydir\num_layers\tuner0.json


In [32]:
tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

In [33]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [34]:
model = tuner.get_best_models(num_models = 1)[0]

C:\Users\prajwal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [35]:
model.fit(X_train, y_train, epochs = 300, initial_epoch = 11, validation_data = (X_test, y_test))

Epoch 12/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7988 - loss: 0.4631 - val_accuracy: 0.8182 - val_loss: 0.4820
Epoch 13/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7905 - loss: 0.4405 - val_accuracy: 0.8052 - val_loss: 0.4781
Epoch 14/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8404 - loss: 0.3961 - val_accuracy: 0.8182 - val_loss: 0.4734
Epoch 15/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8408 - loss: 0.3905 - val_accuracy: 0.7727 - val_loss: 0.5474
Epoch 16/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8175 - loss: 0.4168 - val_accuracy: 0.8247 - val_loss: 0.4856
Epoch 17/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8347 - loss: 0.3773 - val_accuracy: 0.7792 - val_loss: 0.4724
Epoch 18/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8391 - loss: 0.3755 - val_accuracy: 0.8052 - val_loss: 0.4914
Epoch 19/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8534 - loss: 0.3515 - val_accuracy: 0

In [36]:
# Now Applying multiple things

In [37]:
def build_model(hp):
    
    model = Sequential()
    counter = 0

    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
        
        if counter == 0:
            
            model.add(Dense(hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                            activation = hp.Choice('activation' + str(i), values = ['relu', 'tanh', 'sigmoid']),
                            input_dim = 8
                           )
                     )
        else:
            model.add(Dense(hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                            activation = hp.Choice('activation' + str(i), values = ['relu', 'tanh', 'sigmoid'])
                           )
                     )
            counter += 1

        model.add(Dense(1, activation = 'sigmoid'))
        model.compile(optimizer = hp.Choice('optimizer', values = ['rmsprop', 'adam', 'sgd', 'adam', 'adadelta']),
                      loss = 'binary_crossentropy', 
                      metrics = ['accuracy']
                     )
        return model

In [38]:
tuner = kt.RandomSearch(build_model,
                       objective = 'val_accuracy',
                       max_trials = 3,
                       directory = 'my_dir',
                       project_name = 'final')

Reloading Tuner from my_dir\final\tuner0.json


In [39]:
tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

In [40]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10, 'units0': 32, 'activation0': 'tanh', 'optimizer': 'rmsprop'}

In [41]:
model = tuner.get_best_models(num_models = 1)[0]

In [43]:
model.fit(X_train, y_train, epochs = 300, initial_epoch = 11, validation_data = (X_test, y_test))

Epoch 12/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7256 - loss: 0.5103 - val_accuracy: 0.7987 - val_loss: 0.4999
Epoch 13/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7861 - loss: 0.4803 - val_accuracy: 0.8052 - val_loss: 0.4915
Epoch 14/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7435 - loss: 0.4948 - val_accuracy: 0.7857 - val_loss: 0.4855
Epoch 15/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7655 - loss: 0.4679 - val_accuracy: 0.7727 - val_loss: 0.4810
Epoch 16/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7573 - loss: 0.4823 - val_accuracy: 0.7727 - val_loss: 0.4768
Epoch 17/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7581 - loss: 0.4592 - val_accuracy: 0.7792 - val_loss: 0.4730
Epoch 18/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7762 - loss: 0.4635 - val_accuracy: 0.7727 - val_loss: 0.4730
Epoch 19/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7628 - loss: 0.4708 - val_accuracy: 0